In [3]:
import facebook    #sudo pip install facebook-sdk
import json
import pandas as pd
import datetime
import numpy as np

# Token de acesso a página

In [6]:
access_token = "XXX"
user = 'XXX' # Nome de Identificação da Fanpage

In [7]:
graph = facebook.GraphAPI(access_token)
profile = graph.get_object(user)


In [8]:
dfMessages = { 'Comment': [], 'Classificação': []}
dfMessages = pd.DataFrame(data=dfMessages)

# Faz uma lista de todos os posts de acordo com o ano de inicio

In [9]:
allposts = graph.get_all_connections(id=profile['id'],
                                     connection_name='posts',
                                     since=datetime.date(2020,1,1)
                                     )

# Busca os comentários de cada post

In [10]:
for postid in allposts:
    comentario = graph.get_connections(id=postid['id'], connection_name='comments')
    for i in comentario['data']:
        dfMessages = dfMessages.append({'Comment': i['message'], 'Classificação': 'Neutro'}, ignore_index=True)

# Grava os comentário num arquivo CSV (Temporário)

In [11]:
dfMessages.to_csv('test.csv', index = False)

In [12]:
dfMessages

,Comment,Classificação
0,ABENÇOADA PASCOA. BE BLESS. 💐❤️,Neutro
1,🙏🐰,Neutro
2,Bom dia,Neutro
3,E uma pena não encontrar esses produtos nos me...,Neutro
4,👏😋♥️,Neutro
...,...,...
74,,Neutro
75,Boa tarde. Muito bom,Neutro
76,Bom dia,Neutro
77,ABENÇOADO ANO NOVO 👏💃🎇🎆🎉🎊 BE BLESS 💃🎇🎆🎉🎊,Neutro


# Processo de Análise de Sentimento em cima dos comentários recolhidos da fanpage

In [14]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [15]:
from yandex.Translater import Translater
tr = Translater()
tr.set_key('xxx') # Api key found on https://translate.yandex.com/developers/keys
tr.set_from_lang('pt')
tr.set_to_lang('en')

In [16]:
for index, row in dfMessages.iterrows():
    sSentimento = 'Neutro'
    
    # Caso o comentário não esteja vazio
    if row['Comment']:
    
        sComment = row['Comment']
        tr.set_text(sComment)
        sComment = tr.translate()
        
        vs = analyzer.polarity_scores(sComment)
    
        if vs['compound'] >= 0.05:
            sSentimento = 'Positivo'
        elif vs['compound'] <= -0.05:
            sSentimento = 'Negativo'

    # Acrescenta-se a classificação no dataframe original
    dfMessages.loc[index]['Classificação'] = sSentimento
    print(row['Comment'], ' ', sSentimento)
        
    

    


ABENÇOADA PASCOA. BE BLESS. 💐❤️   Positivo
🙏🐰   Neutro
Bom dia   Positivo
E uma pena não encontrar esses produtos nos mercados daqui   Negativo
👏😋♥️   Neutro
Será que tem bacalhau ou é  tipo a coxinha que não  tem frango? 🤨😬   Negativo
Seja qual for a combinação são extremamente ótimos 😋♥️   Positivo
Boa tarde   Positivo
Gostaria de saber se é  padrão vir sem recheio ou comprei um pacote com problema de fábrica?   Negativo
Péssima coxinha, só vem massa. 🤔
Deveria chamar "Bolinha de massa"
Não aconselho ninguém  comprar.
Ainda bem que  comprei só 1 pacote 😡🤬   Negativo
❤️   Neutro
😋😋♥️   Neutro
Que Deus abençoe todos  vocês.   Positivo
Parabéns a toda equipe maricota! Que Deus abençoe vocês e suas famílias   Positivo
   Neutro
👏👏🙏   Neutro
👏🏻👏🏻   Neutro
Eu amo o pão de queijo de vcs !!!!!!   Positivo
   Neutro
Parabéns, Maricota!!!!!    Positivo
👏   Neutro
👏👏🙏   Neutro
👏👏👏👏   Neutro
Parabéns ,tenho orgulho te ter feito parte deste empresa!!!!Que caía bençãos sobre a empresa e seus admin

In [17]:
dfMessages.to_csv('classificacao.csv', index = False)